# Wrapper Functions & Decorators — Practice Notebook

This notebook walks through wrapper functions step‑by‑step, then finishes with the `@decorator` syntax.

## 1) Understand a wrapper function (basic)
A *wrapper function* is a function that defines and returns another function (often called `inner`). The inner function usually does **extra work** before/after calling the original function.

Goal: print `this is wrapper` every time our wrapped function runs.

In [5]:
def wrapper(func):
    def inner():
        print("this is wrapper")
        func()
    return inner

def greet():
    print("hello!")

wrapped_greet = wrapper(greet)
wrapped_greet()  # Expect: prints 'this is wrapper' then 'hello!'


this is wrapper
hello!


## 2) Wrapper to measure execution time
We can add timing logic around `func()` using `time.perf_counter()`.

In [ ]:
import time

def time_wrapper(func):
    def inner():
        start = time.perf_counter()
        try:
            retval = func()
            return retval
        finally:
            end = time.perf_counter()
            print(f"time: {end - start:.6f} seconds")
    return inner

def slow_work():
    total = 0
    for i in range(10_000_00):  # ~1e6 loop
        total += i
    return total

timed_slow_work = time_wrapper(slow_work)
result = timed_slow_work()
print("result:", result)


## 3) Pass parameters through the wrapper
Use `*args` and `**kwargs` to forward any positional/keyword arguments to the wrapped function.

In [ ]:
def wrapper_with_args(func):
    def inner(*args, **kwargs):
        print("this is wrapper (args forwarded)")
        retval = func(*args, **kwargs)
        return retval
    return inner

def add(a, b, scale=1):
    return scale * (a + b)

wrapped_add = wrapper_with_args(add)
print(wrapped_add(3, 4))               # 7
print(wrapped_add(3, 4, scale=10))     # 70


## 4) Return a value from the inner function
`inner` should return the result of `func(*args, **kwargs)` so callers receive the original function's value.

In [ ]:
def wrapper_returns_value(func):
    def inner(*args, **kwargs):
        print("this is wrapper (will return value)")
        value = func(*args, **kwargs)
        print("wrapped function returned:", value)
        return value  # <-- return to caller
    return inner

def multiply(a, b):
    return a * b

wrapped_multiply = wrapper_returns_value(multiply)
out = wrapped_multiply(6, 7)
print("final:", out)


## 5) Decorator (`@`) syntax
Decorators are just a nicer way to apply a wrapper. `@decorator_name` above a function is equivalent to `func = decorator_name(func)`.

### The first decorator example

In [ ]:
def wrapper(func):
    def inner():
        print("this is wrapper")
        func()
    return inner

def hello():
    print("hello!")

wrapped_hello = wrapper(hello)
wrapped_hello()

In [6]:
def simple_decorator(func):
    def inner():
        print("this is wrapper")
        return func()
    return inner

@simple_decorator
def hello():
    print("hello!")

hello()

this is wrapper
hello!


### 2) Decorator Example 2


In [11]:
import time
from functools import wraps

def decorator(func):
    @wraps(func)  # keeps original __name__, __doc__, etc.
    def inner(*args, **kwargs):
        print("this is wrapper (decorator)")
        start = time.perf_counter()
        try:
            return func(*args, **kwargs)
        finally:
            end = time.perf_counter()
            print(f'Elapsed: {end - start:.6f} seconds')
    return inner

@decorator
def fib(n):
    """Return nth Fibonacci number (naive)."""
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

print("fib(10) ->", fib(10))

this is wrapper (decorator)
this is wrapper (decorator)
this is wrapper (decorator)
this is wrapper (decorator)
this is wrapper (decorator)
this is wrapper (decorator)
this is wrapper (decorator)
this is wrapper (decorator)
this is wrapper (decorator)
this is wrapper (decorator)
Elapsed: 0.000001 seconds
this is wrapper (decorator)
Elapsed: 0.000000 seconds
Elapsed: 0.000044 seconds
this is wrapper (decorator)
Elapsed: 0.000000 seconds
Elapsed: 0.000066 seconds
this is wrapper (decorator)
this is wrapper (decorator)
Elapsed: 0.000000 seconds
this is wrapper (decorator)
Elapsed: 0.000000 seconds
Elapsed: 0.000014 seconds
Elapsed: 0.000135 seconds
this is wrapper (decorator)
this is wrapper (decorator)
this is wrapper (decorator)
Elapsed: 0.000000 seconds
this is wrapper (decorator)
Elapsed: 0.000000 seconds
Elapsed: 0.000014 seconds
this is wrapper (decorator)
Elapsed: 0.000000 seconds
Elapsed: 0.000028 seconds
Elapsed: 0.000179 seconds
this is wrapper (decorator)
this is wrapper (decor

### (Optional) Combine multiple decorators
Order matters: the top decorator runs last around the function call.

In [12]:
from functools import wraps

def logger(func):
    @wraps(func)
    def inner(*args, **kwargs):
        print(f"[logger] calling {func.__name__}{args, kwargs}")
        value = func(*args, **kwargs)
        print(f"[logger] returned {value}")
        return value
    return inner

def ensure_int(func):
    @wraps(func)
    def inner(*args, **kwargs):
        value = func(*args, **kwargs)
        if not isinstance(value, int):
            raise TypeError("return value must be int")
        return value
    return inner

@ensure_int
@logger
def sum_to(n):
    return sum(range(n+1))

print("sum_to(5) ->", sum_to(5))


[logger] calling sum_to((5,), {})
[logger] returned 15
sum_to(5) -> 15


## Exercises
1. Modify `wrapper_with_args` to also print the keyword arguments.
2. Write a decorator `@count_calls` that counts how many times a function has been called and prints the count.
3. Write a decorator `@retry(k)` that retries a function up to `k` times if it raises an exception.
4. Create a decorator that caches results for a pure function (like `fib`). Compare with `functools.lru_cache`.